In [1]:
import pandas as pd
import time
import numpy as np

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import InvalidSelectorException

from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
daraz_url = "https://www.daraz.pk/"    

In [3]:
driver.get(daraz_url)
electronics = driver.find_element(By.XPATH,'/html/body/div[1]/div/div/div[2]/div/div/div/div/div/div/div/ul/li[7]/a/span[2]')
electronics.click()

In [4]:
phones = driver.find_element(By.XPATH,'/html/body/div[1]/div/div/div[2]/div/div/div/div/div/div/div/ul/ul[7]/li[1]')
phones.click()

In [5]:
product_id = []
company = []
product_name = []
price = []
ratings = []
product_link = []
specs = []


review_count = []
question_count = []
product_idd = []
total_stars = []
total_review = []
total_review_id = []
overall = []
review_name = []
company__name = []

stars2 = []
review2 = []
reviewid2 = []

In [9]:
exclude = ['case','charger','cover']
id = 0

pages_to_scrap = 5

for page in range(1):

    cards = driver.find_elements(By.CLASS_NAME,'gridItem--Yd0sa')

    for card in cards:
    
        details = card.text.split('\n')
        check = details[0]
        
        if not any(word in check.lower() for word in exclude): 
            id = id + 1
            product_name.append(check)
            company_name = check.split()[0]
            company.append(company_name)
            price.append(details[1])
            product_id.append(id)

            link = card.find_element(By.TAG_NAME, "a")
            link = link.get_attribute("href")
            product_link.append(link)

            driver.execute_script("window.open('');")
            driver.switch_to.window(driver.window_handles[1])
            driver.get(link)

            try:
                review__count = driver.find_element(By.XPATH,'/html/body/div[4]/div/div[3]/div[2]/div/div[1]/div[4]/div[1]/div/div/a[1]')
                review__count = review__count.text.split(' ')[0]
                review_count.append(review__count)
                try:
                    question = driver.find_element(By.XPATH,'/html/body/div[4]/div/div[3]/div[2]/div/div[1]/div[4]/div[1]/div/div/a[2]')
                    questions = question.text.split(' ')[0]
                    question_count.append(questions)
                except NoSuchElementException:
                    question_count.append('-') 
            except NoSuchElementException:
                review_count.append('-') 
                
            try:
                driver.execute_script('window.scrollTo(0, 700) ')
                time.sleep(1)
                rating = driver.find_element(By.XPATH, "/html/body/div[4]/div/div[9]/div[1]/div[1]/div/div/div/div[2]/div[1]/div[1]/div[1]")
                rating = rating.text.split()
                rating = rating[0]
                ratings.append(rating)
            except NoSuchElementException:
                ratings.append('-')

            try:
                speccs = driver.find_element(By.XPATH,'/html/body/div[4]/div/div[9]/div[1]/div[2]/div/div/div[1]/div[2]/ul/li')
                specss = speccs.text.split('\n')
                specs.append(specss)
            except NoSuchElementException:
                try:
                    speccs = driver.find_element(By.XPATH,'/html/body/div[4]/div/div[9]/div[1]/div[2]/div/div/div[1]/div[1]/ul/li')
                    specss = speccs.text.split('\n')
                    specs.append(specss)
                except NoSuchElementException:
                    specs.append('-')
                
            pos = 0
            neg = 0
            nuetral = 0

            review = []
            review_id = []
            stars = []
            count1 = 0
            
            for i in range(pages_to_scrap):
                time.sleep(2)
                driver.execute_script("window.scrollTo(0, 500);")
                time.sleep(1)
                driver.execute_script("window.scrollTo(500, 800);")
                time.sleep(1)
                driver.execute_script("window.scrollTo(800, 1000);")
                time.sleep(2)
                
                try:
                    grid = driver.find_element(By.XPATH, '/html/body/div[4]/div/div[9]/div[1]/div[1]/div/div/div/div[3]')
                except NoSuchElementException:
                    break

                review_box = grid.find_elements(By.CLASS_NAME,'review-item')
                if len(review_box)==0:
                    count1 = count1 + 1
                    company__name.append(company_name)
                    review_name.append(count1)
                    product_idd.append(id)
                    review_id.append('-')
                    reviewid2.append('-')
                    review.append('No Reviews')
                    review2.append('No Reviews')
                    break
                else:
                    for x in review_box:
                        count = 0
                        user = x.find_element(By.CLASS_NAME,'review-content-fl')
                        reviews = x.find_element(By.CLASS_NAME, 'review-content-sl')
                        starss = x.find_element(By.CLASS_NAME,'container-star ')
                        starss = starss.find_elements(By.TAG_NAME, "img")
        
                        if reviews.text != "":
                            users = user.find_element(By.CLASS_NAME,'user').text
                            company__name.append(company_name)
                            count1 = count1 + 1
                            review_name.append(count1)
                            product_idd.append(id)
                            review_id.append(users)
                            reviewid2.append(users)
                            review.append(reviews.text)
                            review2.append(reviews.text)
                            
                            sentiment_scores = analyzer.polarity_scores(reviews.text)
                            if sentiment_scores['compound'] >= 0.05:
                                sentiment = 'Positive'
                                overall.append(sentiment)
                            elif sentiment_scores['compound'] <= -0.05:
                                sentiment = 'Negative'
                                overall.append(sentiment)
                            else:
                                sentiment = 'Neutral'
                                overall.append(sentiment)

                            for star in starss:
                                link = star.get_attribute("src")
                                if (link=='https://laz-img-cdn.alicdn.com/tfs/TB19ZvEgfDH8KJjy1XcXXcpdXXa-64-64.png'):
                                    count = count + 1 
                            stars.append(count)
                            stars2.append(count)    
                            
                    if(i<pages_to_scrap-1):
                        try:
                            button = driver.find_element(By.CLASS_NAME,'ant-pagination-next')
                            button.click()
                        except NoSuchElementException:
                            break
                            
            total_review.append(review)
            total_review_id.append(review_id)
            total_stars.append(stars)

            driver.close()

            driver.switch_to.window(driver.window_handles[0])
            print("Product: ",id)
        
    if page< 4:
        next_button = driver.find_element (By.CSS_SELECTOR, 'li[title="Next Page"]')
        next_button.click ()
        time.sleep (3)
        
       

In [1]:
Data = {'Product ID':product_id,'Company': company,'Product Name': product_name,'Price': price,
        'Rating':ratings,'Total Ratings':review_count,'Total Questions Answered':question_count,'Specifications':specs,'Link':product_link}
Mobile_Data = pd.DataFrame(Data)
Mobile_Data = Mobile_Data.set_index('Product ID')
Mobile_Data.to_csv('Final_Phone_Data.csv',index=True)


In [3]:
Mobile_Data = pd.read_csv("Final_Phone_Data.csv",index_col='Unnamed: 0')
Mobile_Data

,Product ID,Company,Product Name,Price,Rating,Total Ratings,Total Questions Answered,Specifications,Link
0,1,Samsung,Samsung A14 4GB/64GB - 5000 mAh Battery - 6.6'...,"Rs. 34,999",4.4,188,336,"[""• Samsung A14 with 4GB/64GB memory.• 5000 mA...",https://www.daraz.pk/products/samsung-a14-4gb6...
1,2,Redmi,Redmi Note 12 8 GB RAM + 128GB ROM PTA Approve...,"Rs. 49,999",4.7,401,-,['Redmi Note 12 8GB RAM + 128GB ROM PTA Approv...,https://www.daraz.pk/products/redmi-note-12-8-...
2,3,Infinix,Infinix Note 30 Pro 8-256 GB PTA Approved With...,"Rs. 64,499",4.7,152,217,"['Release Date 2023-05-22', 'SIM Support Dual ...",https://www.daraz.pk/products/infinix-note-30-...
3,4,Redmi,Redmi A2+ - 3GB/64GB - Mediatek Helio G36 - An...,"Rs. 19,399",4.7,367,-,['Display: 6.52” HD+ Dot Drop displayResolutio...,https://www.daraz.pk/products/redmi-a2-3gb64gb...
4,5,Realme,Realme Narzo 50A Prime (4GB-128GB) PTA Approve...,"Rs. 32,999",4.2,93,141,"[""'Colour"", 'Xanh Tia Chop = Lightning Blue', ...",https://www.daraz.pk/products/realme-narzo-50a...
...,...,...,...,...,...,...,...,...,...
195,196,Infinix,"Infinix Smart 7 HD , 2GB RAM 64GB ROM , 8MP Ma...","Rs. 21,499",5.0,1,20,['General FeaturesRelease Date2023-04-14SIM Su...,https://www.daraz.pk/products/7-hd-2gb-64gb-8m...
196,197,Tecno,Tecno Spark 10 RAM 4 GB ROM 128 GB Front Camer...,"Rs. 34,999",5.0,3,-,['BuildOSAndroid 13 OS UIHios 12.0 DimensionsN...,https://www.daraz.pk/products/10-4-gb-128-gb-8...
197,198,Infinix,Infinix Zero 30 4G RAM 8 GB ROM 256 GB Front C...,"Rs. 69,999",5.0,1,-,['BuildOSAndroid 13 OS UIXOS 13 Dimensions164....,https://www.daraz.pk/products/30-4g-8-gb-256-g...
198,199,Tecno,"Tecno Camon 20 Pro - 6.67"" Inch Display - 8GB ...","Rs. 61,899",-,No,-,['1 Year Brand Warranty'],https://www.daraz.pk/products/tecno-camon-20-p...


In [21]:
Data = {'Product ID': product_idd,'Review ID':review_name,'Company':company__name,'Review':review2,'Stars':stars2,'Overall Review':overall}
Review = pd.DataFrame(Data)
Review = Review.set_index('Product ID')
Review.to_csv('Final_Review_Data.csv',index=True)



In [10]:
Review_Data = pd.read_csv("Final_Review_Data.csv",index_col='Unnamed: 0')
Review_Data

,Product ID,Review ID,Company,Review,Stars,Overall Review
0,1,1,Samsung,Unbelievable! phone is 100 percent original an...,5,Positive
1,1,2,Samsung,"Genuine samsung mobile, well packed, pta appro...",5,Positive
2,1,3,Samsung,"#11.11 Mobile without mobile warranty card, ch...",5,Positive
3,1,4,Samsung,"Genuine samsung mobile, well packed, pta appro...",5,Positive
4,1,5,Samsung,"#11.11 Mobile without mobile warranty card, ch...",5,Positive
...,...,...,...,...,...,...
1647,198,4,Infinix,"Original product, fast delivery, seller is ver...",5,Positive
1648,198,5,Infinix,"Original product, fast delivery, seller is ver...",5,Positive
1649,200,1,ZTE,"Great packing, working well, original, satisfi...",5,Positive
1650,200,2,ZTE,"Great packing, working well, original, satisfi...",5,Positive
